# Classifying Stability of Mantle with TensorFlow

The Stability of Mantle is defined by Rayleigh number

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
#When using Jupyter notebook make sure to call tf.reset_default_graph() 
# at the beginning to clear the symbolic graph before defining new nodes.
tf.reset_default_graph()

In [3]:
#Define a function to generate data set
def generate_data_set(instance_num = 10000, split_rate = 0.6):
    #instance[0]  Gravitational acceleration
    #instance[1]  Volume expansion coefficient
    #instance[2]  Kinematic viscosity coefficient 
    #instance[3]  Thermal diffusivity
    #instance[4]  Depth
    #instance[5]  b
    #instance[6] λ
    #instance[7] (T0 - T1)/1000
    
    #label (1,0) for stable (0,1) for unstable
    
    #instance[8] stability 0 is unstable and 1 is stable
    counter_for_stable = 0
    counter_for_unstable = 0
    
    data_set = {'input':np.zeros([instance_num, 8]),'label':np.zeros([instance_num,2])}
    
    #simulate gravitational accelerations
    data_set['input'][:,0] = np.random.uniform(0.8,1.0,size=instance_num)
    
    #simulate Volume expansion coefficient
    data_set['input'][:,1] = np.random.uniform(1e-4,1e-2,size=instance_num)
    
    #simulate Kinematic viscosity coefficient
    data_set['input'][:,2] = np.random.uniform(1e-2,1.0,size=instance_num)
    
    #simulate Thermal diffusivity
    data_set['input'][:,3] = np.random.uniform(0.1,1.0,size=instance_num)
    
    #simulate Depth 10000km
    data_set['input'][:,4] = np.random.uniform(0,0.35,size=instance_num)
    
    #simulate b 10000km
    for idx in range(instance_num):    
        data_set['input'][idx,5] = np.random.uniform(max([0.25,data_set['input'][idx,4]]),0.35)
    
    #simulate λ
    data_set['input'][:,6] = np.random.uniform(0.0,0.39,size=instance_num)
    
    #simulate T0 - T1
    data_set['input'][:,7] = np.random.uniform(0.0,0.5,size=instance_num)
    
    for idx in range(instance_num):
        #
        g = data_set['input'][idx,0]*10
        a = data_set['input'][idx,1]
        v = data_set['input'][idx,2]*1000
        k = data_set['input'][idx,3]*10000
        d = data_set['input'][idx,4]*10000
        b = data_set['input'][idx,5]*10000
        lam = data_set['input'][idx,6]*10000
        dT = data_set['input'][idx,7]*1000
        
        Ra = (a*g*dT*(d**3))/(v*k)
        
        Racr = (np.pi**4*((4+(lam/b)**2)**3))/(4*((lam/b)**4))
        
        if Ra > Racr:
            data_set['label'][idx,0] = 0
            data_set['label'][idx,1] = 1
            counter_for_unstable += 1
        else:
            data_set['label'][idx,0] = 1
            data_set['label'][idx,1] = 0
            counter_for_stable += 1
    split_index = int(instance_num*split_rate)
    
    train_set = {'input':data_set['input'][0:split_index,:],
                 'label':data_set['label'][0:split_index,:]}
    test_set = {'input':data_set['input'][split_index:,:],
                 'label':data_set['label'][split_index:,:]}
    
    print('Stable:{} UnStable:{}'.format(counter_for_stable, counter_for_unstable))
    
    return train_set,test_set

In [4]:
data_set_size = 1000000
split_rate = 0.5
train_set, test_set = generate_data_set(instance_num = data_set_size, split_rate = split_rate)

Stable:574480 UnStable:425520


In [5]:
#define full connection layer
def full_connection_layer(input_tensor, n_out, 
                          w_init=tf.truncated_normal_initializer(stddev=0.1), 
                          b_init=tf.constant_initializer(0.1), 
                          activation=tf.nn.sigmoid, name=None):
    n_in = input_tensor.get_shape().as_list()[1]
    with tf.variable_scope(name):
        weight = tf.get_variable('weight',[n_in, n_out],initializer=w_init)
        bias = tf.get_variable('bias',[n_out],initializer=b_init)
    output_tensor = activation(tf.matmul(input_tensor,weight)+bias,name=name+'_output')
    return output_tensor

In [6]:
def inference(input_tensor):
    hidden_layer_1 = full_connection_layer(input_tensor=input_tensor,n_out=4, name='fc_layer_1')
    hidden_layer_2 = full_connection_layer(input_tensor=hidden_layer_1,n_out=4, name='fc_layer_2')
    hidden_layer_3 = full_connection_layer(input_tensor=hidden_layer_2,n_out=4, name='fc_layer_3')
    pred = full_connection_layer(input_tensor=hidden_layer_3,n_out=2, name='pred')
    return pred

In [7]:
#set param for training
step_num = 20001
batch_size = 1000
data_length = 8
learning_rate = 0.01
#setup training 
input_tensor = tf.placeholder(tf.float32,[None,data_length], name='input')
label = tf.placeholder(tf.float32,[None,2], name='label')
pred = inference(input_tensor=input_tensor)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=label))
train_op = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

In [8]:
def get_traning_batch(train_set, batch_size, data_length):
    batch_ids = np.random.choice(len(train_set['input']),batch_size)
    input_batch = np.zeros([batch_size,data_length])
    label_batch = np.zeros([batch_size,2])
    for idx in range(batch_size):
        input_batch[idx][:] = train_set['input'][batch_ids[idx]][:]
        label_batch[idx][:] = train_set['label'][batch_ids[idx]][:]
    return input_batch,label_batch

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [10]:
#start traning
for idx in range(step_num):
    input_batch, label_batch = get_traning_batch(train_set, batch_size, data_length)
    _, loss_val = sess.run([train_op, loss], {input_tensor: input_batch, label: label_batch})
    if idx%2000 == 0:
        print(loss_val)

0.6934938
0.39525732
0.3629702
0.3573489
0.35604522
0.34598345
0.34709588
0.34188947
0.354488
0.3468753
0.3486143


In [11]:
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(label,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
print('Accuracy: {}'.format(sess.run(accuracy,{input_tensor: test_set['input'], label: test_set['label']})))

Accuracy: 0.9649959802627563
